In [2]:
import pandas as pd

In [3]:
df=pd.read_csv("simulated_pos_data_with_seasonal_trends.csv")
df.head(15)

,Timestamp,Customer_ID,Customer_Name,Item_Category,Item_Name,Quantity,Price_Per_Item,Total_Price,Payment_Method
0,2023-06-21 10:35:45,518529,Cassidy David,Cool Drinks,son,4,17.86,71.44,Cash
1,2023-04-23 19:13:55,929362,Kayla Ball,Fruits,commercial,5,7.46,37.30,Credit Card
2,2023-08-13 17:13:43,390753,James Vaughn,Tea,bed,1,8.81,8.81,Debit Card
3,2024-01-24 18:38:37,310934,Eric Spears,Curries,fine,1,12.62,12.62,Credit Card
4,2023-04-09 14:24:55,781110,Hayley King,Ice Cream,her,5,17.69,88.45,Cash
5,2023-09-25 11:22:34,671615,Edward Case,Coffee,property,1,10.58,10.58,Debit Card
6,2023-05-06 04:37:27,421259,Deborah Graham,Cool Drinks,of,5,5.58,27.90,Credit Card
7,2023-04-30 05:44:24,233961,Jacqueline Johnson,Ice Cream,degree,5,18.40,92.00,Cash
8,2024-03-15 11:32:01,711114,Kenneth Reyes,Ice Cream,space,3,16.27,48.81,Credit Card
9,2024-03-17 09:14:04,584205,Stefanie Davis,Ice Cream,morning,5,6.11,30.55,Cash


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Timestamp       1000 non-null   object 
 1   Customer_ID     1000 non-null   int64  
 2   Customer_Name   1000 non-null   object 
 3   Item_Category   1000 non-null   object 
 4   Item_Name       1000 non-null   object 
 5   Quantity        1000 non-null   int64  
 6   Price_Per_Item  1000 non-null   float64
 7   Total_Price     1000 non-null   float64
 8   Payment_Method  1000 non-null   object 
dtypes: float64(2), int64(2), object(5)
memory usage: 70.4+ KB


In [17]:
# Convert Timestamp column to datetime
df['Timestamp'] = pd.to_datetime(df['Timestamp'], errors='coerce')

# Create a dimension table from unique dates
unique_dates = pd.Series(df['Timestamp'].dt.date.unique()).sort_values()
dim_date = pd.DataFrame({'date': unique_dates})

# Extract date features
dim_date['year'] = pd.to_datetime(dim_date['date']).dt.year
dim_date['month'] = pd.to_datetime(dim_date['date']).dt.month
dim_date['day'] = pd.to_datetime(dim_date['date']).dt.day
dim_date['day_of_week'] = pd.to_datetime(dim_date['date']).dt.day_name()
dim_date['quarter'] = pd.to_datetime(dim_date['date']).dt.quarter

# Add a date_id
dim_date.reset_index(drop=True, inplace=True)
dim_date['date_id'] = dim_date.index + 1

In [18]:
# Create payment method dimension table
unique_methods = df['Payment_Method'].dropna().unique()
dim_payment_method = pd.DataFrame({'payment_method': sorted(unique_methods)})
dim_payment_method['payment_method_id'] = dim_payment_method.index + 1

In [19]:
# Create item dimension table
dim_item = df[['Item_Category', 'Item_Name', 'Price_Per_Item']].drop_duplicates()
dim_item.columns = ['item_category', 'item_name', 'price']
dim_item = dim_item.reset_index(drop=True)
dim_item['item_id'] = dim_item.index + 1

In [20]:
# Create customer dimension table
dim_customer = df[['Customer_ID', 'Customer_Name']].drop_duplicates()
dim_customer.columns = ['customer_id', 'customer_name']
dim_customer = dim_customer.reset_index(drop=True)

In [21]:
# Build fact_sales
fact = df.copy()
# link to date_id
fact['date'] = fact['Timestamp'].dt.date
fact = fact.merge(dim_date[['date','date_id']], on='date', how='left')
# link to item_id
fact = fact.merge(
    dim_item[['item_category','item_name','item_id']],
    left_on=['Item_Category','Item_Name'],
    right_on=['item_category','item_name'],
    how='left'
)
# link to payment_method_id
fact = fact.merge(
    dim_payment_method[['payment_method','payment_method_id']],
    left_on='Payment_Method',
    right_on='payment_method',
    how='left'
)

fact_sales = (
    fact.rename(columns={
            'Customer_ID':'customer_id',
            'Quantity':'quantity',
            'Price_Per_Item':'price_per_item',
            'Total_Price':'total_price'
        })
        .loc[:, [
            'Timestamp','date_id','customer_id',
            'item_id','payment_method_id',
            'quantity','price_per_item','total_price'
        ]]
)

In [23]:
dim_date.to_csv(f'dim_date.csv', index=False)
dim_payment_method.to_csv(f'dim_payment_method.csv', index=False)
dim_item.to_csv(f'dim_item.csv', index=False)
dim_customer.to_csv(f'dim_customer.csv', index=False)
fact_sales.to_csv(f'fact_sales.csv', index=False)